# Getting started 
I will assume that since this is open you at the very least have anaconda installed... and from this you will have some packages called numpy, matplotlib, and scipy, installed 
Now when you run any of the code lines bellow you will need to select a python enviroment to run it in the command

## Using a package
Even though these packages are installed they won't be inside your enviorment (the memory used for executing code)
To bring them into your enviroment you will need to import them
Lets start off by importing two of the most important packages and exploring what they can do for us.

In [1]:
import numpy as np 
import matplotlib.pyplot as plt 

The as command just gives the package an alias thats quicker to type. 
## Numpy 
or numerical python is a package that handles matrices and numerical calculations in python
## matplotlib 
is a package that allows you to plot figures and is   